In [1]:
import numpy
def LAT(inputMask, outputMask):
    count=0
    
    for a in range(len(sBox)):
        key=dec(a)
        value=dec(sBox[a])
        
        left=inputMask&key
        right=value&outputMask
        left=bin(left)[2:].zfill(4)
        right=bin(right)[2:].zfill(4)
        
        left=XOR(left)
        right=XOR(right)
        if left==right:
            count+=1
    
    count = count-pow(2,3)
    return count
        
        
def XOR(x):
    if len(x)==1:
        return x
    else:
        x=str(int(x[0])^int(x[1])) + x[2:]
        return XOR(x)    

def dec(x):
    if type(x)==int:
        return x
    else:
        if x=='A' or x=='a':
            return 10
        if x=='B' or x=='b':
            return 11
        if x=='C' or x=='c':
            return 12
        if x=='D' or x=='d':
            return 13
        if x=='E' or x=='e':
            return 14
        if x=='F' or x=='f':
            return 15

In [2]:
def DDT(inputMask,outputMask):
    count=0
    for a in range(len(sBox)):
        key=dec(a)
        value=dec(sBox[a])
        left_right=dec(sBox[key^inputMask])
        left=value^left_right
        if left==outputMask:
            count+=1
    return count

In [3]:
def LAT_calculate(sBox):
    latTable = numpy.zeros((16,16))
    for i in range(len(sBox)):
        for j in range(len(sBox)):
            latTable[i][j]=LAT(i,j)
    print('-'*100)        
    print('LAT TABLE')
    print('-'*100 + '\n')  
    print(latTable)
    print('\n')
    return latTable
    
def DDT_calculate(sBox):
    ddtTable = numpy.zeros((16,16))
    for i in range(len(sBox)):
        for j in range(len(sBox)):
            ddtTable[i][j]=DDT(i,j)

    print('-'*100)        
    print('DDT TABLE')
    print('-'*100 + '\n')  
    print(ddtTable)
    print('\n')
    return ddtTable
    
def LAT_nonlinearity(sBox):
    absLAT=numpy.absolute(LAT_calculate(sBox))
    absLAT[0][0]=0
    NLMs=8-numpy.max(absLAT)
    nonLinear=NLMs/6*100
    print('\n' + '-'*100)       
    print('LAT NON-LINEARITY = %'+str(nonLinear))
    print('-'*100 + '\n')

In [4]:
while True:
    qr = input('S-box değerlerini programın hesaplamasını ister misiniz? (istemezseniz S-box değerlerini manuel girmelisiniz.)')
    if qr == '' or not qr[0].lower() in ['e','h']:print('Lütfen evet veya hayır cevabı giriniz')
    else:break
if qr[0].lower() == 'e': 
    sBox = [x for x in input("S-box değerlerini arada boşluk bırakarak giriniz: ").split()] 
    #print("", x)
if qr[0].lower() == 'h':
    print('hey')

S-box değerlerini programın hesaplamasını ister misiniz? (istemezseniz S-box değerlerini manuel girmelisiniz.)e
S-box değerlerini arada boşluk bırakarak giriniz: 0 1 f a 8 6 5 9 4 7 3 e d c b 2


In [5]:
print("GF(2^4)'de indirgenemez polinomlar aşağıda verilmiştir.\n")
print('x^4+x+1\nx^4+x^3+x^2+x+1\nx^4+x^3+1')

GF(2^4)'de indirgenemez polinomlar aşağıda verilmiştir.

x^4+x+1
x^4+x^3+x^2+x+1
x^4+x^3+1


In [6]:
# S-box values
#sBox=[0,1,'f','a',8,6,5,9,4,7,3,'e','d','c','b',2]

In [7]:
LAT_calculate(sBox)

TypeError: unsupported operand type(s) for &: 'NoneType' and 'int'

In [ ]:
#DDT_calculate(sBox)

In [ ]:
#LAT_nonlinearity(sBox)